In [1]:
# libraries necessary for data acquisition and processing (mandatory)
import requests
import h5py
import matplotlib.pyplot as plt
import numpy as np
# libraries necessary for creating gifs of subhalo evolution (optional)
from PIL import Image
import io


baseurl = 'http://www.illustris-project.org/api/'
headers = {"api-key":"API_KEY_HERE"} # <- put your own Illustris API key here


def get(path, params=None):

    r = requests.get(path, params=params, headers=headers)
    r.raise_for_status()

    if r.headers['content-type'] == 'application/json':
        return r.json()
    

    if 'content-disposition' in r.headers:
        filename = r.headers['content-disposition'].split("filename=")[1]
        with open(filename, 'wb') as f:
            f.write(r.content)
        return filename
    
    return r


In [2]:
# We're creating dark matter density histograms, so these are the only fields I need to extract from each subhalo snapshot
cutout_request = {'dm':'Coordinates,SubfindDensity'}

# Starting from the first subhalo of the last snapshot, change relevant numbers in the link to track a different subhalo
link = 'https://www.illustris-project.org/api/Illustris-3/snapshots/135/subhalos/0/'


i = 135
frames = []

while True:

    sub = get(link)
    sub_cutout = get(link + '/cutout.hdf5')
    i = sub['snap']
    print('Snapshot: ' + str(i))

    
    # Creating a y-z histogram, (un)comment respective lines to change
    
    with h5py.File(sub_cutout, 'r') as f:
        #x = f['PartType1']['Coordinates'][:,0] - sub['pos_x']
        y = f['PartType1']['Coordinates'][:,1] - sub['pos_y']
        z = f['PartType1']['Coordinates'][:,2] - sub['pos_z']
        wgt = np.log10(f['PartType1']['SubfindDensity'][:])


    # plotting
    plt.figure(figsize=(6,6))
    plt.hist2d(y, z, weights=wgt,bins=[150,150], cmap='RdBu')
    plt.title(f"Snapshot {i}")
    plt.xlabel('Y [ckpc/h]')
    plt.ylabel('Z [ckpc/h]')

    # saving plot to memory
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    # moving the plot to the frames list
    frames.append(Image.open(buf))
    plt.close()

    
    candidate = sub['related']['sublink_progenitor']

    # if the progenitor link is marked as null, the current subhalo is the first progenitor and we end the loop
    if candidate != None:
        link = candidate
    else:
        break

Snapshot: 135
Snapshot: 134
Snapshot: 133
Snapshot: 132
Snapshot: 131
Snapshot: 130
Snapshot: 129
Snapshot: 128
Snapshot: 127
Snapshot: 126
Snapshot: 125
Snapshot: 124
Snapshot: 123
Snapshot: 122
Snapshot: 121
Snapshot: 120
Snapshot: 119
Snapshot: 118
Snapshot: 117
Snapshot: 116
Snapshot: 115
Snapshot: 114
Snapshot: 113
Snapshot: 112
Snapshot: 111
Snapshot: 110
Snapshot: 109
Snapshot: 108
Snapshot: 107
Snapshot: 106
Snapshot: 105
Snapshot: 104
Snapshot: 103
Snapshot: 102
Snapshot: 101
Snapshot: 100
Snapshot: 99
Snapshot: 98
Snapshot: 97
Snapshot: 96
Snapshot: 95
Snapshot: 94
Snapshot: 93
Snapshot: 92
Snapshot: 91
Snapshot: 90
Snapshot: 89
Snapshot: 88
Snapshot: 87
Snapshot: 86
Snapshot: 85
Snapshot: 84
Snapshot: 83
Snapshot: 82
Snapshot: 81
Snapshot: 80
Snapshot: 79
Snapshot: 78
Snapshot: 77
Snapshot: 76
Snapshot: 75
Snapshot: 74
Snapshot: 73
Snapshot: 72
Snapshot: 71
Snapshot: 70
Snapshot: 69
Snapshot: 68
Snapshot: 67
Snapshot: 66
Snapshot: 65
Snapshot: 64
Snapshot: 63
Snapshot: 62
Sn

In [3]:
# create gif with reverse order of frames (ascending time)
frames.reverse()
frames[0].save('evolution.gif', save_all=True, append_images=frames[1:], duration=200, loop=0)

print('done')

done
